In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://www.kaggle.com/datasets/aradhakkandhari/suicidal-mental-health-dataset
https://www.kaggle.com/datasets/suchintikasarkar/sentiment-analysis-for-mental-health

In [ ]:
data_location = "/content/drive/MyDrive/Colab_Notebooks/final_project/data.csv"
data1_location = "/content/drive/MyDrive/Colab_Notebooks/final_project/data1.csv"

In [ ]:
import pandas as pd

data = pd.read_csv(data_location, index_col=[0])
data1 = pd.read_csv(data1_location)

In [ ]:
data

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...
53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [ ]:
data1

,text,label
0,I recently went through a breakup and she said...,depression
1,"I do not know how to navigate these feelings, ...",depression
2,"So I have been with my bf for 5 months , and h...",depression
3,I am so exhausted of this. Just when I think I...,SuicideWatch
4,I have been severly bullied since i was 5 till...,depression
...,...,...
20359,I took 50mg of seroquel a few hours after I dr...,SuicideWatch
20360,that is what has happened to me last week. And...,depression
20361,Ever just feel alone in a house full of people...,depression
20362,Politicians. Neighbors. Corporations. Society....,depression


In [ ]:
print(data['status'].value_counts())

status
Normal                  16351
Depression              15404
Suicidal                10653
Anxiety                  3888
Bipolar                  2877
Stress                   2669
Personality disorder     1201
Name: count, dtype: int64


In [ ]:
labels = ["Normal", "Suicidal"]

In [ ]:
# examples
import random
for label in labels:
  print(data[data['status'] == label].values[random.randint(0,len(data[data['status'] == label]))])

["no, they don't. one is black and one is dark blue." 'Normal']
['Does anybody else feel better after imagining themselves resorting to suicide when dealing with stressful situations? I never truly plan to go through with it (not yet at least), however the thought bouncing in my head provides a much needed sense of relief as morbid as it may seem. Question'
 'Suicidal']


In [ ]:
data = data.dropna()

In [ ]:
data = data[data['status'].isin(["Normal", "Suicidal", "Depression"])]

In [ ]:
data['status'] = data['status'].apply(lambda x: 1 if x == 'Suicidal' else 0)

In [ ]:
data1['label'] = data1['label'].apply(lambda x: 0 if x == 'depression' else 1)

In [ ]:
data1 = data1.rename(columns={'label': 'status', 'text': 'statement'})

In [ ]:
data = pd.concat([data, data1])

In [ ]:
data

,statement,status
733,"Gr gr dreaming of ex crush to be my game, God",0
734,wkwkwk what a joke,0
735,Leaves are also standby in front of the PC ......,0
736,Thank God even though it's just a ride through,0
737,wedding teaser concept using the song day6 - o...,0
...,...,...
20359,I took 50mg of seroquel a few hours after I dr...,1
20360,that is what has happened to me last week. And...,0
20361,Ever just feel alone in a house full of people...,0
20362,Politicians. Neighbors. Corporations. Society....,0


In [ ]:
print(data['status'].value_counts())

status
0    42118
1    20645
Name: count, dtype: int64


In [ ]:
rows = data[data['status'] == 0]
rows = rows.sample(n=int(data[data['status'] == 1].shape[0]), random_state=42)

In [ ]:
rows

,statement,status
39642,i m a yo guy and i want to die i feel like a b...,0
38739,hi so can anxiety stress make a person physica...,0
14104,I do not have any friends for 2 year i lonely ...,0
40184,i lost everything lost friend lost the love of...,0
9133,"I think that is the quote at least, correct me...",0
...,...,...
4700,"Dr. Tiktok, Amanda Elli looks like April Kepner",0
1583,what are you doing skks,0
18065,"Why is everything so darn heavy, even the air ...",0
8410,I am just some bitter asshole.Gunna be 20 soon...,0


In [ ]:
data = pd.concat([data[data['status'] == 1], rows])

In [ ]:
print(data['status'].value_counts())

status
1    20645
0    20645
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['statement'].tolist(), data['status'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)
test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128
)


In [ ]:
import torch

class DepressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], dtype=torch.long) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = DepressionDataset(train_encodings, train_labels)
test_dataset = DepressionDataset(test_encodings, test_labels)

In [ ]:
print(type(train_encodings['input_ids'][0]))
print(train_encodings['input_ids'][0][:10])

<class 'list'>
[101, 2002, 2003, 5458, 1012, 102, 0, 0, 0, 0]


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    save_strategy="epoch",
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-236-974de8259d31>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
trainer.train() # when using the gpu runtime in colab the trainer should automaticly detect the cude interface.

Epoch,Training Loss,Validation Loss
1,0.341100,0.392667
2,0.254800,0.325116
3,0.042600,0.403776


TrainOutput(global_step=6195, training_loss=0.29506856051301456, metrics={'train_runtime': 2376.1769, 'train_samples_per_second': 41.704, 'train_steps_per_second': 2.607, 'total_flos': 6518313285488640.0, 'train_loss': 0.29506856051301456, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import classification_report

preds_output = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(preds_output.predictions), axis=1)

print(classification_report(test_labels, preds))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88      4106
           1       0.87      0.91      0.89      4152

    accuracy                           0.89      8258
   macro avg       0.89      0.89      0.89      8258
weighted avg       0.89      0.89      0.89      8258



In [ ]:
trainer.save_model("./bert-depression-model")
tokenizer.save_pretrained("./bert-depression-model")

('./bert-depression-model/tokenizer_config.json',
 './bert-depression-model/special_tokens_map.json',
 './bert-depression-model/vocab.txt',
 './bert-depression-model/added_tokens.json')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
def predict(text):
  inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
)
  inputs = {key: val.to(device) for key, val in inputs.items()}
  print("Tokens: " + str(inputs))
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    return labels[predicted_class_id]

In [ ]:
labels

['Normal', 'Suicidal']

In [ ]:
print(predict("How can i take my own life"))

Tokens: {'input_ids': tensor([[ 101, 2129, 2064, 1045, 2202, 2026, 2219, 2166,  102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Suicidal


In [ ]:
import torch
labels = ["Normal", "Suicide"]
def predict_from_pretained(model, tokenizer, text):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  model.eval()
  inputs = tokenizer(
    text,
    return_tensors="pt",
    truncation=True,
    padding=True,
    max_length=128
)
  inputs = {key: val.to(device) for key, val in inputs.items()}
  print("Tokens: " + str(inputs))
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()
    return labels[predicted_class_id]

In [ ]:
import transformers
model = transformers.BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab_Notebooks/final_project/model', num_labels=2)
tokenizer = transformers.BertTokenizer.from_pretrained('/content/drive/MyDrive/Colab_Notebooks/final_project/tokenizer')

In [ ]:
print(predict_from_pretained(model, tokenizer, "i want to jump off a building"))

Tokens: {'input_ids': tensor([[ 101, 1045, 2215, 2000, 5376,  102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}
Normal
